# IoT Challenge - Geolocalization

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from geopy.distance import vincenty

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

In [3]:
# load train and test data
df_mess_train = pd.read_csv('mess_train_list.csv') # train set
df_mess_test = pd.read_csv('mess_test_list.csv') # test set
pos_train = pd.read_csv('pos_train_list.csv') # position associated to train set

## Data exploration [Théo]

## Outliers processing I [André]

Following our realization of outlier bases with geolocalization positions that do not seem to make sense, we decided to compute, approximatelly, their longitude and latitude by using the coordinates of the (weighted) barycentre of the messages each outlier base received.

The code coded provided below did this cleaning

In [5]:
# List of unique messages
listOfmess = np.unique(df_mess_train['messid'])

# DataFrame with all training_data, including positions
df = pd.concat([df_mess_train, pos_train], axis=1)

In [19]:
# We can notice that the outlier bases have latitude at 64.3 and longitude at -68.5:
df.groupby(['bsid']).mean()[['bs_lat', 'bs_lng']].sort_values(['bs_lat'], ascending=False).head(10)

bs_lat  bs_lng
bsid                 
1772     64.3   -68.5
4156     64.3   -68.5
8560     64.3   -68.5
2943     64.3   -68.5
8449     64.3   -68.5
4987     64.3   -68.5
11951    64.3   -68.5
2293     64.3   -68.5
7248     64.3   -68.5
9784     64.3   -68.5

In [45]:
# Selecting these bases
bases_out = df[(df['bs_lat']==64.3) & (df['bs_lng']==-68.5)]['bsid'].unique()
bases_out

array([ 8355, 11007,  1594, 10151, 10162,  8451,  4993,  8560,  2293,
        4959, 10999,  1661,  8449,  4156,  4129,  1743,  4987,  1772,
        1796,  2707,  2943,  4123, 11951,  9784,  1092,  1854,  7248])

In [70]:
# Getting dataframe with all data for the ouliers bases
df_out = df[df['bsid'].isin(bases_out)]

# Initiating arrays that will have the lat, long and rssi of messages received by the bases, 
# Each column represents a message
mess_num = len(listOfmess) # number of messages
lat_array = np.zeros((df_out.shape[0], mess_num))
lng_array = np.zeros((df_out.shape[0], mess_num))
weight_array = np.zeros((df_out.shape[0], mess_num)) # weights to be used: sqrt(exp(rssi))
    
# Dictionary to track message id and corresponding column in array
mess_dict = {}
for i, column in enumerate(listOfmess):
    mess_dict[column] = i
    
# assigning values to arrays
for i, ix in enumerate(df_out.index):
    mess = df_out.loc[ix, 'messid']
    column = mess_dict[mess]
    
    # Using sqrt(exp(rssi)) as weight to get weighted centroid
    weight_array[i, column] = np.sqrt(np.exp(df_out.loc[ix, 'rssi']))
    weight = np.sqrt(np.exp(df_out.loc[ix, 'rssi']))
    lat_array[i, column] = df_out.loc[ix, 'lat'] * weight
    lng_array[i, column] = df_out.loc[ix, 'lng'] * weight
    
# Transforming arrays in dataframe in order to use groupby()
lat_df = pd.DataFrame(lat_array)
lng_df = pd.DataFrame(lng_array)
weight_df = pd.DataFrame(weight_array)

# Adding column bsid for each dataframes in order to perform groupby()
lat_df['bsid'] = lng_df['bsid'] = weight_df['bsid'] = df_out.reset_index()['bsid']

# Grouping and suming --- Note that values for lat and lng are already weighted
lat_df_grouped = lat_df.groupby('bsid').sum()
lng_df_grouped = lng_df.groupby('bsid').sum()
weight_df_grouped = weight_df.groupby('bsid').sum()

# Dividing each row by the sum of the weights for the respective row
lat_df_grouped = lat_df_grouped.divide(weight_df_grouped.sum(axis=1), axis=0)
lng_df_grouped = lng_df_grouped.divide(weight_df_grouped.sum(axis=1), axis=0)

# Getting the final weighted latitudes and longitudes
lat_out = lat_df_grouped.sum(axis=1)
lng_out = lng_df_grouped.sum(axis=1)

# Assigning these new latitudes and longitudes to the bases in the test and training sets
for base in lat_out.index:
    df_mess_train.loc[df_mess_train['bsid']==base, 'bs_lat'] = lat_out.loc[base]
    df_mess_train.loc[df_mess_train['bsid']==base, 'bs_lng'] = lng_out.loc[base]
    
    df_mess_test.loc[df_mess_test['bsid']==base, 'bs_lat'] = lat_out.loc[base]
    df_mess_test.loc[df_mess_test['bsid']==base, 'bs_lng'] = lng_out.loc[base]

## Features Engineering [Mathieu]

### One-Hot (Teacher ++ / Mean / RSSI / etc.. )

### Usage of barycentre

#### Standard barycentre

#### Weighted barycentre

## Model [Mathieu / Matyas / André]

### First raw predictions

### Outliers processing II

### Fine-tunning / Model selection

### Blending of models predictions

In [ ]:
def regressor_and_predict_devices_XtraTree(df_feat, ground_truth_lat, ground_truth_lng, df_test, **kwargs):
    '''
    Train regressor and make prediction in the train set
    Input: df_feat: feature matrix used to train regressor
           ground_truth_lat: df_feat associated latitude
           ground_truth_lng: df_feat associated longitude
           df_test: data frame used for prediction
    Output: y_pred_lat, y_pred_lng
    '''
    
    X = df_feat
    y_lat = ground_truth_lat
    y_lng = ground_truth_lng
    
    model = ExtraTreesRegressor(n_jobs=-1, max_depth=3000, n_estimators=500,
                                min_samples_split=6, min_impurity_decrease=2.1544346900318822e-07)
    
    model.fit(X, np.array([y_lat, y_lng]).T)
    y_pred_lat, y_pred_lng = model.predict(df_test).T
    
    return y_pred_lat, y_pred_lng

In [ ]:
def regressor_and_predict_devices_xgb_lat(df_feat, ground_truth_lat, df_test, **kwargs):
    '''
    Train regressor and make prediction in the train set
    Input: df_feat: feature matrix used to train regressor
           ground_truth_lat: df_feat associated latitude
           ground_truth_lng: df_feat associated longitude
           df_test: data frame used for prediction
    Output: y_pred_lat, y_pred_lng
    '''
    
    X = df_feat
    y_lat = ground_truth_lat
    
    model = xgb.XGBRegressor(n_estimators=1000, n_jobs=-1)
    #print(model)

    model.fit(X, np.array([y_lat]).T)
    y_pred_lat = model.predict(df_test).T
    
    
    return y_pred_lat



def regressor_and_predict_devices_xgb_lng(df_feat, ground_truth_lng, df_test, **kwargs):
    '''
    Train regressor and make prediction in the train set
    Input: df_feat: feature matrix used to train regressor
           ground_truth_lat: df_feat associated latitude
           ground_truth_lng: df_feat associated longitude
           df_test: data frame used for prediction
    Output: y_pred_lat, y_pred_lng
    '''
    
    X = df_feat
    y_lng = ground_truth_lng
    model = xgb.XGBRegressor(n_estimators=1000, n_jobs=-1)
    #print(model)
    model.fit(X, np.array([y_lng]).T)
    y_pred_lng = model.predict(df_test).T
    
    
    return y_pred_lng

In [3]:
kf = KFold(n_splits=n_devices_train, shuffle=False, random_state=0)

devices_out = []
devices_far = set([])


k_fold_perc_fusion = []
k_fold_perc_xtra = []
k_fold_perc_xgb = []
for step, (train_index, test_index) in enumerate(kf.split(devices)):
        
    train_devices = devices[train_index]
    test_devices = devices[test_index]
    devices_out.append(test_devices[0])
        
    mess_train = df_mess_train[df_mess_train.did.isin(train_devices)].messid.unique()
    mess_test = df_mess_train[df_mess_train.did.isin(test_devices)].messid.unique()

    X_train = df_feat.loc[mess_train]
    X_test = df_feat.loc[mess_test]
        
    lat_train, lng_train = (ground_truth_lat[df_feat.index.isin(mess_train)], 
                            ground_truth_lng[df_feat.index.isin(mess_train)])
    lat_test, lng_test = (ground_truth_lat[df_feat.index.isin(mess_test)],
                            ground_truth_lng[df_feat.index.isin(mess_test)])
        
        
    # ExtraTree predictions
    y_pred_lat_xtra_lat, y_pred_xtra_lng = regressor_and_predict_devices_XtraTree(X_train, lat_train, lng_train, X_test, 
                                                               **best_params)
        
    # XGBoost predictions
    y_pred_xgb_lat = regressor_and_predict_devices_xgb_lat(X_train, lat_train, X_test, 
                                                               **best_params)
    y_pred_xgb_lng = regressor_and_predict_devices_xgb_lng(X_train, lng_train, X_test, 
                                                               **best_params)
        
    # Mean of the predictions
    y_pred_lat = (y_pred_lat_xtra_lat + y_pred_xgb_lat) / 2
    y_pred_lng = (y_pred_xtra_lng + y_pred_xgb_lng) / 2
    
    err_xtra = Eval_geoloc(lat_test, lng_test, y_pred_lat_xtra_lat, y_pred_xtra_lng)
    err_xgb = Eval_geoloc(lat_test, lng_test, y_pred_xgb_lat, y_pred_xgb_lng)
    err_fusion = Eval_geoloc(lat_test, lng_test, y_pred_lat, y_pred_lng)
    
    percentile_fusion = np.percentile(err_fusion, 80)
    k_fold_perc_fusion.append(percentile_fusion)
    
    percentile_xtra = np.percentile(err_xtra, 80)
    k_fold_perc_xtra.append(percentile_xtra)
    
    percentile_xgb = np.percentile(err_xgb, 80)
    k_fold_perc_xgb.append(percentile_xgb)
    
    if(percentile_fusion > 10000):
        devices_far.add(test_devices[0])
        
    print(f"Crossvalidation step {step}.. device {test_devices[0]}.. percentile XTree: {percentile_xtra:.2f}")
    print(f"Crossvalidation step {step}.. device {test_devices[0]}.. percentile XGB: {percentile_xgb:.2f}")
    print(f"Crossvalidation step {step}.. device {test_devices[0]}.. percentile fusion: {percentile_fusion:.2f}")
    print("-----------------------------------------------------")
    
mean_perc_fusion = np.array(k_fold_perc_fusion).mean()
mean_perc_xtra = np.array(k_fold_perc_xtra).mean()
mean_perc_xgb = np.array(k_fold_perc_xgb).mean()
print("-----------------------------------------------------")
print(f"Percentile mean for fusion {mean_perc_fusion:.2f}")
print(f"Percentile mean for XTree {mean_perc_xtra:.2f}")
print(f"Percentile mean for XGB {mean_perc_xgb:.2f}")
print("-----------------------------------------------------\n")
#percentiles.append(mean_perc)

#best_param = params_list[np.array(percentiles).argmin()]


## Testset preprocessing [André] 

We can notice that the test set has some of the outliers bases that are not present in the training set:

In [93]:
out_bases_test = df_mess_test[(df_mess_test['bs_lat']==64.3) & (df_mess_test['bs_lng']==-68.5)]['bsid'].unique()
out_bases_test

array([9949, 9941])

We can not replace their positions as we do not have the coordinates of the messages in the test set. Therefore, we decided to not include these two bases when computing the features (weighted barycentre) of the messages.

There is however one of these two bases, base 9949, that detect two messages which are only detected by that base:

In [91]:
# Getting the messages detected by the base 9949
messages_9949 = df_mess_test[df_mess_test.bsid == 9949].messid.values
messages_9949

array(['57b99c16cf554f465ad8de48', '57b9eff912f1434591626c19'],
      dtype=object)

In [92]:
# Display all rows with both messages
df_mess_test[df_mess_test.messid.isin(messages_9949)]

messid  bsid       did  nseq        rssi  \
13431  57b99c16cf554f465ad8de48  9949  472066.0   2.0 -122.000000   
13434  57b9eff912f1434591626c19  9949  472066.0   1.0 -118.666667   

            time_ux  bs_lat  bs_lng  
13431  1.471782e+12    64.3   -68.5  
13434  1.471803e+12    64.3   -68.5

We can see that this does not happen with the base 9941 as each of the three messages it detects is detected by the base 9941:

In [94]:
messages_9941 = df_mess_test[df_mess_test.bsid == 9941].messid.values
df_mess_test[df_mess_test.messid.isin(messages_9941)]

messid  bsid       did  nseq        rssi  \
14667  57cbf92412f1437531983238  9936  472066.0   1.0 -118.333333   
14668  57cbf92412f1437531983238  9941  472066.0   1.0 -123.500000   
14669  57cbfbbfcf554f22dc736cb2  9936  472066.0   1.0 -111.333333   
14670  57cbfbbfcf554f22dc736cb2  9941  472066.0   1.0 -124.666667   
28670  5843e9a6cf554f422f2b7495  9936  472066.0   1.0 -127.500000   
28671  5843e9a6cf554f422f2b7495  9941  472066.0   2.0 -135.000000   

            time_ux     bs_lat      bs_lng  
14667  1.472985e+12  48.072889 -110.957181  
14668  1.472985e+12  64.300000  -68.500000  
14669  1.472986e+12  48.072889 -110.957181  
14670  1.472986e+12  64.300000  -68.500000  
28670  1.480846e+12  48.072889 -110.957181  
28671  1.480846e+12  64.300000  -68.500000

AS both messages detected by the base 9949 are from the same device (did == 472066.0) we decided to predict their position as the mean position of the predictions of other detected messages from the same device. 

### Processing test set for the same structure as training set

In [96]:
# List of bases for test, excluding bases 9949, 9941
listOfBs_test = np.setdiff1d(
                    np.union1d(np.unique(df_mess_train['bsid']), np.unique(df_mess_test['bsid'])),
                    out_bases_test
                )

In [112]:
# Feature Matrix construction for test
# There is a try / except cell to handle keyError with bsids 9949 and 9941

def feat_mat_const_test(df_mess_test, listOfBs):
    ''' 
    Method used for creation of features. It uses numpy for calculations due to optimization issues
    
    Parameters
    ----------
    df_mess_test: raw test dataFrame with input data
    listOfBs: list of unique bases present in both train and test sets
    
    Returns
    -------
    df_feat: modified dataFrame with created features: longitude and latitude of the (weighted) 
    barycentre of the bases that detected each message
    '''
    
    bases_num = len(listOfBs)
    
    # Dictionnary that will track the column in the numpy array and its correpondent base 
    bases_dict = {}
    for i, column in enumerate(listOfBs):
        bases_dict[column] = i
    
    # Creating array to be transformed at feature matrix (columns = features)
    lat_array = np.zeros((df_mess_test.shape[0], bases_num))
    lng_array = np.zeros((df_mess_test.shape[0], bases_num))
    weights_array = np.zeros((df_mess_test.shape[0], bases_num))
    
    # assigning values for new features
    for i in df_mess_test.index:
        try: 
            # getting bsid for current row
            bsid = df_mess_test.loc[i, 'bsid']
            # getting colum to assign value in bases_array
            column = bases_dict[bsid]
            # assigning weight (sqrt(rssi)) value to cell in bases array
            weight = np.exp(df_mess_test.loc[i, 'rssi'])**(1/2)
            lat_array[i, column] = weight * df_mess_test.loc[i, 'bs_lat']
            lng_array[i, column] = weight * df_mess_test.loc[i, 'bs_lng']
            weights_array[i, column] = weight
        
        # To handle when we get a key error when bsid is equal to 9941 or 9949
        except KeyError: 
            pass
    
    # Tranforming bases_array to dataFrame in order to use .groupby()

    lat_df = pd.DataFrame(lat_array)
    lng_df = pd.DataFrame(lng_array)
    weights_df = pd.DataFrame(weights_array)
    
    
    # Using groupby() and getting final dataFrame of features
    
    # Adding messid column to each data-frame (lat, lng and weights)
    lat_df['messid'] = df_mess_test['messid']
    lng_df['messid'] = df_mess_test['messid']
    weights_df['messid'] = df_mess_test['messid']
    
    lat = lat_df.groupby('messid').sum().sum(axis=1)
    lng = lng_df.groupby('messid').sum().sum(axis=1)
    weights = weights_df.groupby('messid').sum().sum(axis=1)
    
    # Normalizing
    df_feat = pd.DataFrame([lat, lng], index=['lat','lng'])
    df_feat = (df_feat/weights).T
    
    return df_feat

In [106]:
# Creating dataframe of features for the test set
df_test = feat_mat_const_barycentre(df_mess_test, listOfBs_test)

In [107]:
df_test.head()

lat         lng
messid                                         
573be2503e952e191262c351  39.728651 -105.163032
573c05f83e952e1912758013  39.783207 -105.088708
573c0796f0fe6e735a66deb3  39.655285 -105.043437
573c08d2864fce1a9a0563bc  39.782113 -105.072701
573c08ff864fce1a9a0579b0  39.655282 -105.043385

In [110]:
# Checking for the messages detected by base 9949 (predictions to be replaced)
df_test.loc[messages_9949]

lat  lng
messid                            
57b99c16cf554f465ad8de48  NaN  NaN
57b9eff912f1434591626c19  NaN  NaN

In [111]:
# Checking for the messages detected by base 9941 (these will not be replaced)
df_test.loc[messages_9941]

lat         lng
messid                                         
57cbf92412f1437531983238  48.072889 -110.957181
57cbfbbfcf554f22dc736cb2  48.072889 -110.957181
5843e9a6cf554f422f2b7495  48.072889 -110.957181

## Running final model and computing predictions on test set [André] 